# TAIL RISK FUND: OPTIONS (BHANSALI STRATEGY)

author: [@sparshsah](https://github.com/sparshsah)

ref: https://github.com/sparshsah/foggy-demo/blob/main/demo/finance/tail-risk-fund_options.ipynb

In [1]:
from __future__ import annotations

import dataclasses
import datetime
from typing import Final, Literal

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# https://github.com/sparshsah/foggy-statslib
import sys; sys.path.append("../../../foggy-statslib/")
import foggy_statslib.core as fsc
import foggy_statslib.fin as fsf

In [17]:
del ROLL_BUFFER_DAYS, IMPL_LAG_DAYS

In [7]:
START_DATE: Final[datetime.datetime] = datetime.datetime(2014, 7, 1)
PREVAILING_INTEREST_RATE: Final[float] = 


# UTILS


def _calc_vol(r: pd.Series[float], freq: str = "B") -> pd.Series[float]:
    """Hackily calculate realized vol from returns."""
    if r.index.freq != "B":
        raise ValueError(freq)
    return (
        (
            r
            # assert that ER_daily is close to zero
            .abs()
            # hackily remove influence of stock splits
            .clip(upper=0.20)
            **2
        )
        # a business quarter
        .rolling(window=65, min_periods=1)
        .mean()
        **0.5
        # annualize
        * 261**0.5
    ).rename("vol")


@dataclasses.dataclass(kw_only=True, frozen=True)
class StrategyConfig:
    """By default, buy COF puts quarterly at 20% OTM, selling iff value hits 2x initial price."""
    budget_dol: float = 100
    underlying_ticker: str = "COF"
    period_freq: Literal["M", "Q", "2Q", "Y"] = "Q"
    roll_buffer_days: int = 5  # look to roll this many trading days before expiration
    strike_otm_px: float = 0.20
    sell_thresh: float = 2.0
    impl_lag_days: int = 3  # trading lag in days

    @property
    def periods(self) -> pd.PeriodIndex:
        """Get the backtest periods."""
        return pd.period_range(
            start=START_DATE,
            end=datetime.datetime.now(),
            freq=self.period_freq,
            name="period",
        # first period is burn-in
        )[1:]

    @property
    def timeseries(self) -> pd.DataFrame:
        """Get the backtest periods, and associated data (e.g. entry/exit dates)."""
        ts = pd.DataFrame(index=self.periods)
        ts.loc[:, "entry_date"] = ts.index.start_time - ROLL_BUFFER_DAYS * pd.offsets.BDay()
        ts.loc[:, "expiration_date"] = ts.index.end_time
        return ts


class MarketDataAccess:
    """Module for getting data."""

    @staticmethod
    def get_stock_history(ticker: str = "COF") -> pd.DataFrame:
        p = (
            pd.read_csv(
                f"data/yahoo-finance_NYSE-{ticker}_px.csv",
                index_col="Date",
                parse_dates=True,
            )
            .loc[START_DATE:, "Close"]
            .rename_axis("date").rename("unadj_close_price_dol")
            .asfreq("D").ffill()
        )
        p_b = p.asfreq("B")
        r_b = p_b / p_b.shift() - 1
        vol_b = _calc_vol(r=r_b)
        vol = vol_b.asfreq("D").ffill()
        history = pd.concat([p, vol], axis="columns", verify_integrity=True)
        return history

    @staticmethod
    def _generate_put_histories() -> pd.DataFrame:
        """Manually Black-Scholes just the data points we need given the strategy we're trying to backtest."""
        cfg = StrategyConfig()
        ts = cfg.timeseries
        sh = MarketDataAccess.get_stock_history(ticker=cfg.underlying_ticker)

        def _gen_cell(contract_ticker: pd.Period, t: pd.Timestamp) -> float:
            period = contract_ticker
            entry_date = ts.loc[period, "entry_date"]
            expiration_date = ts.loc[period, "expiration_date"]
            if entry_date <= t <= expiration_date:
                return fsf.calc_bsf_option_value(
                    # set at entry
                    put=True,
                    K=sh.loc[entry_date, "unadj_close_price_dol"] * (1 - cfg.strike_otm_px),
                    # calculated today
                    S_t=sh.loc[t, "unadj_close_price_dol"],
                    sigma=sh.loc[t, "vol"],  # let's assume a 1.1x vol premium
                    tau=(expiration_date - t).days / 365,
                    # whatever
                    r=0.05,
                )
            else:
                return float("nan")

        ph = pd.DataFrame(
            columns=ts.index,
            index=pd.date_range(start=START_DATE, end=sh.last_valid_index(), freq="D", name="date"),
            dtype=float,
        )
        ph = pd.DataFrame(
            {
                period: {
                    t: _gen_cell(contract_ticker=period, t=t)
                    for t in ph.index
                }
                for period in ph.columns
            },
            index=ph.index,
            columns=ph.columns,
        )
        return ph

    @staticmethod
    def get_put_history(
        ticker: pd.Period,
        start_date: datetime.datetime,
        end_date: datetime.datetime,
    ) -> pd.Series[float]:
        history = PUT_HISTORIES.loc[start_date:end_date, ticker]
        # from generated data, this should be valid already
        assert start_date in history.index, (start_date, history.index[:8])
        assert end_date in history.index, (end_date, history.index[-8:])
        assert not history.isna().any(), history.isna().mean()
        # for IRL data, we might need to clean up
        dtx = pd.date_range(start=start_date, end=end_date, freq="D", name="date")
        history = history.reindex(index=dtx).ffill()
        return history


# # DATA


# STOCK_HISTORY = MarketDataAccess.get_stock_history()
# PUT_HISTORIES = MarketDataAccess._generate_put_histories()

In [13]:
cfg = StrategyConfig()
cfg.timeseries.head()

,entry_date,expiration_date
period,,
2014Q4,2014-09-24,2014-12-31 23:59:59.999999999
2015Q1,2014-12-25,2015-03-31 23:59:59.999999999
2015Q2,2015-03-25,2015-06-30 23:59:59.999999999
2015Q3,2015-06-24,2015-09-30 23:59:59.999999999
2015Q4,2015-09-24,2015-12-31 23:59:59.999999999
